In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout

Using TensorFlow backend.


In [ ]:
from google.colab import files
files.upload()

{}

In [ ]:
import makeHistoricalData

num = 0
r = 14
target = 'confirmed'
errors = {}
mean_errors = [] # mean on counties for each h
fixed_covariates = ['age_40_59', 'some_college_or_higher', 'age_80_or_higher', 'less_than_high_school_diploma', 
                    'latitude', 'age_0_14', 'hospital_beds', 'diabetes', 'age_15_39', 'icu_beds', 'high_school_diploma_only', 
                    'Religious', 'airport_distance', 'smokers', 'total_college_population', 'area', 'female-percent', 'age_60_79', 
                    'ventilator_capacity', 'passenger_load', 'median_household_income', 'houses_density', 'party', 'total_population', 
                    'longitude', 'population_density', 'meat_plants']

for h in range(1, 8):
    data = makeHistoricalData.makeHistoricalData(h, r, target, 'mrmr')
    counties = data.county_fips.unique()
    mean_error = 0
    df = pd.DataFrame()

    # model = Sequential()
    # model.add(LSTM(4, return_sequences=True, input_shape=(1, None)))
    # model.add(LSTM(256, dropout=0.2,recurrent_dropout=0.2, return_sequences=True))
    # model.add(LSTM(256, dropout=0.2,recurrent_dropout=0.2, return_sequences=True))
    # model.add(LSTM(128, dropout=0.2,recurrent_dropout=0.2, return_sequences=True))
    # model.add(LSTM(128))
    # model.add(Dense(1, activation='sigmoid'))

    # # opt = tf.keras.optimizers.RMSprop(learning_rate=0.1)
    # model.compile (
    #     loss='mean_squared_error',
    #     optimizer=keras.optimizers.Adam(0.001)
    # )
    counties = [36043, 36055, 36059, 41053, 45003, 50011, 20169, 21163, 22079, 8035]
    for i in range(len(counties)):
        # if counties[i] == 5107:
        #     break
        temporal = data.loc[data['county_fips'] == counties[i]]
        temporal.reset_index(drop=True, inplace=True)

        del temporal['date of day t']
        del temporal['county_name']
        del temporal['state_name']
        del temporal['state_fips']
        del temporal['county_fips']
        temporal = temporal.drop(fixed_covariates, axis = 1)

        totalNumberOfDays = len(temporal)

        # last 14 days are for test dataset and the rest are for train dataset
        train = temporal.head(totalNumberOfDays-28)
        validation = temporal.loc[totalNumberOfDays-28:totalNumberOfDays-15, :]
        test = temporal.tail(14)

        X_train = train.drop('Target', axis=1)
        y_train = train['Target']

        X_val = validation.drop('Target', axis=1)
        y_val = validation['Target']

        X_test = test.drop('Target', axis=1)
        y_test = test['Target']

        ####################################### deleting some first days because of many zeros
        X_train = X_train.iloc[28:]
        y_train = y_train.iloc[28:]
        #######################################

        scalar = preprocessing.MinMaxScaler()

        X_train = X_train.values
        X_train = scalar.fit_transform(X_train)

        X_val = X_val.values
        X_val = scalar.fit_transform(X_val)

        X_test = X_test.values
        X_test = scalar.fit_transform(X_test)

        y_train = y_train.values
        y_train = scalar.fit_transform(y_train.reshape(-1, 1))

        y_val = y_val.values
        y_val = scalar.fit_transform(y_val.reshape(-1, 1))

        # y_test = y_test.values
        # min_max_scaler = preprocessing.MinMaxScaler()
        # y_test = min_max_scaler.fit_transform(y_test.reshape(-1, 1))

        X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
        X_val = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))
        X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

        y_train = y_train.reshape((y_train.shape[0]), )
        y_val = y_val.reshape((y_val.shape[0]), )
        # y_test = y_test.reshape((y_test.shape[0]), )

        print('Working on county_fips = ', counties[i], ' and h = ', h)

        print(X_train.shape, X_val.shape, X_test.shape)
        print(y_train.shape, y_val.shape, y_test.shape)
        
        model = Sequential()
        model.add(LSTM(4, return_sequences=True, input_shape=(1, X_train.shape[2])))  # returns a sequence of vectors of dimension 32
        model.add(LSTM(256, dropout=0.2, return_sequences=True))
        model.add(LSTM(256, dropout=0.2, return_sequences=True))
        model.add(LSTM(128, dropout=0.2, return_sequences=True))
        model.add(LSTM(128))
        model.add(Dense(1, activation='selu'))

        # opt = tf.keras.optimizers.RMSprop(learning_rate=0.1)
        model.compile (
            loss='mean_squared_error',
            optimizer=keras.optimizers.Adam(0.01)
        )

        history = model.fit (
            X_train, y_train,
            epochs=50,
            batch_size=128,
            validation_data=(X_val, y_val),
            verbose=1,
            shuffle=False
        )

        # fig = plt.figure()
        # plt.plot(history.history['loss'], label='train')
        # plt.plot(history.history['val_loss'], label='test')
        # plt.xlabel('Epoch')
        # plt.ylabel('Loss')
        # plt.legend()
        # plt.show()
        # fig_name = 'h=' + str(h) + ', county_fips=' + str(counties[i])
        # fig.savefig(fig_name, dpi=600, quality=100)

        pred = model.predict(X_test)
        pred = scalar.inverse_transform(pred.reshape(-1, 1))
        # y_test = scalar.inverse_transform(y_test.reshape(-1, 1))

        y_test = y_test.to_numpy()
        y_test = y_test.reshape(-1, 1)
        pred = pred.reshape(-1, 1)
        pred = np.round(pred)

        ##################################### storing predictinos in a file
        result = pd.DataFrame()
        result['Prediction'] = pred.tolist()
        result['Actual'] = y_test.tolist()
        file_name = '/content/Results/predictions_'+str(counties[i])+'_'+str(h)+'.csv'
        result.to_csv(file_name, mode='w', index=False)
        #####################################

        pred[pred < 0] = 0
        pred[pred == 0] += 1
        y_test = np.round(y_test)
        y_test[y_test == 0] = 1

        MSE = (np.square(y_test - pred)).mean(axis=None)
        RMSE = np.sqrt(MSE)        
        MAPE = np.mean(np.abs((y_test - pred) / y_test)) * 100

        r2 = r2_score(y_test, pred)

        key = str(counties[i])+'_'+str(h)
        temp_list = [[key, MSE, RMSE, MAPE, r2]]
        df = df.append(pd.DataFrame(temp_list, columns=['county and h', 'MSE','RMSE', 'MAPE', 'adj_r_squared']), ignore_index=True)

        error = mean_squared_error(np.round(y_test), np.round(pred))
        mean_error += error
        key = str(counties[i])+'_'+str(h)
        errors[key] = error

    num += 1
    file_name = '/content/Results/errors_per_county_h='+str(num)+'.csv'
    df.to_csv(file_name, mode='w', index=False)
    mean_errors.append(mean_error / float(len(counties)))

Streaming output truncated to the last 5000 lines.
36/36 [==============================] - 0s 922us/step - loss: 0.0169 - val_loss: 0.1085
Epoch 48/50
36/36 [==============================] - 0s 902us/step - loss: 0.0231 - val_loss: 0.1065
Epoch 49/50
36/36 [==============================] - 0s 898us/step - loss: 0.0157 - val_loss: 0.1021
Epoch 50/50
36/36 [==============================] - 0s 896us/step - loss: 0.0179 - val_loss: 0.0978
Working on county_fips =  36059  and h =  3
(36, 1, 27) (14, 1, 27) (14, 1, 27)
(36,) (14,) (14,)
Train on 36 samples, validate on 14 samples
Epoch 1/50
36/36 [==============================] - 2s 62ms/step - loss: 0.1772 - val_loss: 0.1676
Epoch 2/50
36/36 [==============================] - 0s 966us/step - loss: 0.1457 - val_loss: 0.1170
Epoch 3/50
36/36 [==============================] - 0s 808us/step - loss: 0.1065 - val_loss: 0.0658
Epoch 4/50
36/36 [==============================] - 0s 876us/step - loss: 0.0764 - val_loss: 0.0992
Epoch 5/50
36/36

In [ ]:
!rm -rf Results/

In [ ]:
!mkdir Results

In [ ]:
!zip -r /content/Results.zip /content/Results/

  adding: content/Results/ (stored 0%)
  adding: content/Results/errors_per_county_h=7.csv (deflated 51%)
  adding: content/Results/predictions_20169_3.csv (deflated 72%)
  adding: content/Results/predictions_36059_2.csv (deflated 58%)
  adding: content/Results/predictions_45003_4.csv (deflated 59%)
  adding: content/Results/predictions_45003_2.csv (deflated 66%)
  adding: content/Results/predictions_41053_7.csv (deflated 65%)
  adding: content/Results/predictions_36043_7.csv (deflated 71%)
  adding: content/Results/predictions_22079_3.csv (deflated 54%)
  adding: content/Results/predictions_41053_2.csv (deflated 71%)
  adding: content/Results/predictions_22079_5.csv (deflated 53%)
  adding: content/Results/predictions_45003_7.csv (deflated 60%)
  adding: content/Results/predictions_50011_6.csv (deflated 66%)
  adding: content/Results/predictions_20169_2.csv (deflated 70%)
  adding: content/Results/predictions_36055_7.csv (deflated 65%)
  adding: content/Results/predictions_22079_2.csv